In [3]:
import sys
import time
import os
from deepface import DeepFace
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

def extract_pixels_from_image(image: np.ndarray, facial_area: dict) -> np.ndarray:
    x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
    x = max(0, x)
    y = max(0, y)
    face_crop = image[y:y+h, x:x+w]
    face_crop_float64 = face_crop.astype(np.float64) / 255.0
    return face_crop_float64

def extract_and_expand_faces(img_path: str, margin_ratio: float = 0.0) -> list:
    image = cv2.imread(img_path)
    faces = DeepFace.extract_faces(
        img_path=image,
        detector_backend='retinaface',
        enforce_detection=False
    )

    for face_data in faces:
        facial_area = face_data['facial_area']
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
        margin_x = int(w * margin_ratio)
        margin_y = int(h * margin_ratio)
        facial_area['x'] = x - margin_x
        facial_area['y'] = y - margin_y
        facial_area['w'] = w + 2 * margin_x
        facial_area['h'] = h + 2 * margin_y

    for __face__ in faces:
        __face__['face'] = extract_pixels_from_image(image, __face__['facial_area'])
    
    return faces

def get_people_list(path_people_ref: str) -> list:
    ref_directory = path_people_ref
    list_dict = []
    for person_name in os.listdir(ref_directory):
        person_path = os.path.join(ref_directory, person_name)
        if os.path.isdir(person_path):
            images = [img for img in os.listdir(person_path) if img.endswith(".png")]
            image_paths = [os.path.join(person_path, img).replace("\\", "/") for img in images]
            person_dict = {
                "name": person_name,
                "images": images,
                "images_paths": image_paths
            }
            list_dict.append(person_dict)
    return list_dict

list_dict_people = get_people_list("./data/classroom/ref/")

def recognize_person(analyze_faces: list, ref_face: list, metric_model_name: str = "Facenet512", 
                     metric_distance: str = "cosine", metric_threshold: float = 0.44) -> list:
    if not ref_face or len(ref_face) == 0:
        print("Error: The reference face is empty.")
        return []

    target_face_data = ref_face[0]
    target_face = target_face_data['face']
    array_bgr_target_face = (target_face * 255.0).astype(np.uint8)

    results = []
    total_faces = len(analyze_faces)

    for i, face_data in enumerate(analyze_faces):
        loading = f"\rProcessing... {i + 1}/{total_faces} comparisons ({((i + 1) / total_faces) * 100:.2f}%)"
        print(loading, end="")

        facial_area = face_data['facial_area']
        face = face_data['face']
        array_bgr_face = (face * 255.0).astype(np.uint8)

        result = DeepFace.verify(
            img1_path=array_bgr_target_face,
            img2_path=array_bgr_face,
            detector_backend="skip",
            model_name=metric_model_name,
            distance_metric=metric_distance,
            threshold=metric_threshold,
            enforce_detection=False
        )

        results.append({
            "index": i,
            "distance": result['distance'],
            "threshold": result['threshold'],
            "verified": result['verified'],
            "facial_area": facial_area
        })

    results.sort(key=lambda x: x['distance'])
    return results

def draw_verified_faces(image, result_list_dict, name_person="target"):
    total_faces = len(result_list_dict)
    comparison_counter = 0
    for result in result_list_dict:
        facial_area = result['facial_area']
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']

        if result['verified']:
            color = (0, 255, 0)
            confidence = 1.0 - result['distance']
            label = f"{name_person} ({confidence:.2f}%)"
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        comparison_counter += 1
        progress = (comparison_counter / total_faces) * 100
        print(f"\rProcessing faces... {comparison_counter}/{total_faces} ({progress:.2f}%)", end="")

    print("\nFinished drawing verified faces.")
    return image


In [4]:
   def handle_face_collisions(analyze_faces, ref_face_list, person_name):
    comparison_counter = 0
    total_comparisons = len(analyze_faces) * len(ref_face_list)
    results = []

    for analyze_face in analyze_faces:
        for ref_face in ref_face_list:
            # Convertir las imágenes de las caras en arrays BGR para la comparación
            array_bgr_target_face = ref_face["face"]
            array_bgr_face = analyze_face["face"]

            # Primera comparación con umbral bajo
            initial_result = DeepFace.verify(
                img1_path=array_bgr_target_face,
                img2_path=array_bgr_face,
                detector_backend="skip",
                model_name=metric_model_name,
                distance_metric=metric_distance,
                threshold=initial_threshold,
                enforce_detection=False
            )

            # Si la primera comparación es positiva, se realiza la comparación final
            if initial_result['verified']:
                # Segunda comparación con umbral alto
                final_result = DeepFace.verify(
                    img1_path=array_bgr_target_face,
                    img2_path=array_bgr_face,
                    detector_backend="skip",
                    model_name=metric_model_name,
                    distance_metric=metric_distance,
                    threshold=final_threshold,
                    enforce_detection=False
                )

                # Almacenar el resultado de la comparación final
                if final_result['verified']:
                    results.append({
                        "person": person_name,
                        "index": comparison_counter,
                        "distance": final_result['distance'],
                        "threshold": final_result['threshold'],
                        "verified": final_result['verified'],
                        "facial_area": analyze_face['facial_area'],
                        "confidence": 1.0 - final_result['distance']  # Proporción de confianza
                    })

            # Actualización del contador de comparaciones
            comparison_counter += 1
            # Calculamos el porcentaje de progreso
            progress = (comparison_counter / total_comparisons) * 100
            print(f"\rProcessing... {comparison_counter}/{total_comparisons} comparisons ({progress:.2f}%)", end="")

    print("\nFinished processing all face comparisons.")
    
    # Ordenar los resultados por la distancia (menor es mejor)
    results.sort(key=lambda x: x['distance'])

    return results

def main():
    # Ruta de la imagen de referencia y la lista de imágenes a analizar
    ref_image_path = "./data/classroom/ref/person_1.jpg"  # Ruta de ejemplo
    analyze_image_path = "./data/classroom/analyze/test_image.jpg"  # Ruta de imagen a analizar

    # Cargar las caras de referencia (lista de rostros de personas conocidas)
    ref_face_list = extract_and_expand_faces(ref_image_path, margin_ratio=0.2)

    # Cargar las caras a analizar
    analyze_faces = extract_and_expand_faces(analyze_image_path, margin_ratio=0.2)

    # Realizar la comparación de rostros
    results = handle_face_collisions(analyze_faces, ref_face_list, "person_1")

    # Si se encontraron coincidencias, dibujarlas en la imagen original
    if results:
        image = cv2.imread(analyze_image_path)
        image_with_faces = draw_verified_faces(image, results, "person_1")

        # Guardar la imagen con los rostros verificados
        output_image_path = "./output/test_result.jpg"
        cv2.imwrite(output_image_path, image_with_faces)

        print(f"Processed image saved to {output_image_path}")
    else:
        print("No faces verified.")

if __name__ == "__main__":
    main()

ValueError: img must be numpy array or str but it is <class 'NoneType'>